In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss
from sktime.split import SlidingWindowSplitter, temporal_train_test_split
from sktime.utils import plotting
from sktime.utils.plotting import plot_windows

from probafcst.utils.time import get_forecast_dates

In [ ]:
bikes = pd.read_parquet("../data/bikes.parquet")
load = pd.read_parquet("../data/energy.parquet")

In [ ]:
one_year = 24 * 365
cv = SlidingWindowSplitter(
    fh=np.arange(1, 24 * 7), window_length=one_year, step_length=one_year
)
len(list(cv.split(load)))

In [ ]:
plot_windows(cv, load)

In [ ]:
load = pd.read_parquet("../data/load.parquet")
y_train, y_test = temporal_train_test_split(load, test_size=24 * 7)
fh = ForecastingHorizon(y_test.index, is_relative=False)
forecaster = NaiveForecaster(strategy="mean", sp=24)
# forecaster = StatsForecastAutoARIMA()
forecaster.fit(y_train)

quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]
y_pred = forecaster.predict_quantiles(fh, alpha=quantile_levels)

In [ ]:
columns = [y_pred[i] for i in y_pred.columns]
fig, ax = plotting.plot_series(y_test, *columns)

In [ ]:
loss = PinballLoss(score_average=True)
loss(y_test, y_pred)

In [ ]:
# make a submission

timestamps, forecast_hours = get_forecast_dates(load.index[-1])

forecaster = NaiveForecaster()
forecaster.fit(load)
fh = ForecastingHorizon(forecast_hours, is_relative=True)
y_pred = forecaster.predict_quantiles(fh, alpha=quantile_levels)
y_pred

In [ ]:
from probafcst.utils import create_submission

create_submission(energy_preds=y_pred.values)